## Introduction

In this notebook, we will use LlamaIndex and LLAMA-2 to answer questions based on a youtube Video.



In [1]:
import pandas as pd
import numpy as np

import os
import sys 

from pathlib import Path

### The video is about Chandrayaan-3 and chances of survival on coldest night. 


In [2]:
YOUTUBE_URL="https://www.youtube.com/watch?v=ma9aN2HNDv0" 

### There are two main approaches to reading this youtube video using LLAMA Index
1. Download the youtube video and extract the audio and use https://llamahub.ai/l/file-audio Plugin to transcribe the Audio. This internally uses OpenAI Whishper
2. Directly use https://llamahub.ai/l/youtube_transcript plugin transcript to transcribe the audio. This uses youtube_transcript_api library to transcribe.

In this notebook, we will use the youttube_transcript plugin in LLAMA HUB to read the youtube video



In [3]:
#!pip install youtube_transcript_api -q

In [4]:
#!pip install llama-hub -q

In [3]:
from llama_hub.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(ytlinks=[YOUTUBE_URL])

In [4]:
documents

[Document(id_='2f353da9-09c1-47da-bd93-49ff6a3d5750', embedding=None, metadata={'video_id': 'ma9aN2HNDv0'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='da656380d69dcfc43537eeb26dac9e3bca2c2c685424b6f353eac9a035f9e468', text="[Music]\nlanding on the moon is any country's\ndream and India has accomplished that\nwith an astounding Milestone by finally\nLanding the chandrian 3 mission on the\nmoon successfully in one of the untapped\nlocations where ice and water can be\nfound there but nothing comes without\nchallenges and chandron 3's probes are\nfacing a huge one right now they have to\nsurvive the longest and coldest night of\ntheir lives and the problem is that they\ndon't have eaters to keep them warm well\nthat sounds like a challenge and that is\nwhat we are going to talk about in this\nvideo we will tell you everything you\nneed to know about chandron 3's survival\nchallenge during the lunar night and why\nit is important and challenging

### Index the document
We will use Sentence transformer ``sentence-transformers/all-mpnet-base-v2`` to embed the documents. For the Chat, we will use LLAMA-2 q4 quantised model. 

In [5]:
from llama_index import VectorStoreIndex

from langchain.embeddings import HuggingFaceEmbeddings


In [6]:
model_name = "sentence-transformers/all-mpnet-base-v2"
hf = HuggingFaceEmbeddings(
    model_name=model_name
)



/Users/aiswaryaramachandran/miniforge3/envs/py310-whisper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.llm_predictor import LLMPredictor
from langchain.llms import LlamaCpp

For the LLM,let us load the LLAMA-13B Chat Model with q4 Quantisation.  In MAC,M1 qith q4_0 models - we can use the Metal GPU for inference

In [8]:
llm = LlamaCpp(
    # You can pass in the URL to a GGML model to download it automatically
    #model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin",
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    #model_path="/Users/aiswaryaramachandran/DataScienceArena/llm_mac/llama.cpp-master/models/llama-2-13b-chat.gguf.q8_0.bin",
    model_path="/Users/aiswaryaramachandran/.cache/lm-studio/models/TheBloke/CodeUp-Llama-2-13B-Chat-HF-GGUF/codeup-llama-2-13b-chat-hf.Q8_0.gguf",
    n_gpu_layers=-1,
    n_batch=512,
    temperature=0.1,
    max_tokens=256,
    top_p=1,
    verbose=True, # Verbose is required 
    f16_kv=True,
    n_ctx=4096,
    use_mlock=True,n_threads=4

)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/aiswaryaramachandran/.cache/lm-studio/models/TheBloke/CodeUp-Llama-2-13B-Chat-HF-GGUF/codeup-llama-2-13b-chat-hf.Q8_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q8_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q8_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q8_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q8_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q8_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q8_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q8_0     [  5120, 13824,     1,     1 ]
llama

In [9]:
llm_predictor=LLMPredictor(llm=llm)
embed_model = LangchainEmbedding(hf) 
service_context = ServiceContext.from_defaults(embed_model=embed_model,llm_predictor=llm_predictor)

index = VectorStoreIndex.from_documents(
    documents, 
    service_context=service_context,
)


In [10]:
query_engine=index.as_query_engine()


In [11]:
response=query_engine.query("What is a Lunar Night?")


llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =   184.26 ms /   113 runs   (    1.63 ms per token,   613.27 tokens per second)
llama_print_timings: prompt eval time = 11727.95 ms /  2234 tokens (    5.25 ms per token,   190.49 tokens per second)
llama_print_timings:        eval time =  6465.06 ms /   112 runs   (   57.72 ms per token,    17.32 tokens per second)
llama_print_timings:       total time = 18832.36 ms


In [12]:
print(response)

 A lunar night is a period of time when the Moon does not receive any sunlight because the Sun is on the opposite side of the Earth. During this time, the surface temperature of the Moon can drop to as low as -253°C (-424°F), causing extreme cold stress to spacecraft and scientific instruments left on the lunar surface without heaters. The lunar night lasts for about 14 days, which is the same amount of time it takes for the Moon to complete one rotation on its axis.


In [13]:
response=query_engine.query("Can you mention the key points regarding Chandrayaan 3 mission?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =   240.70 ms /   247 runs   (    0.97 ms per token,  1026.17 tokens per second)
llama_print_timings: prompt eval time =   262.72 ms /    20 tokens (   13.14 ms per token,    76.13 tokens per second)
llama_print_timings:        eval time = 14146.99 ms /   246 runs   (   57.51 ms per token,    17.39 tokens per second)
llama_print_timings:       total time = 15144.11 ms


In [14]:
print(response)



* The Chandrayaan-3 mission is India's third lunar mission.
* It consists of two probes, Vikram (lander) and Pragyan (rover).
* The mission aims to study the topography, mineralogy, surface chemistry, and thermal properties of the lunar South Pole region.
* The probes landed near the lunar South Pole on August 23rd, 2023.
* The mission does not have any heaters on board to survive the extreme temperatures on the moon.
* The probes had to prepare for the lunar night without any heaters by fully charging their batteries, turning off scientific instruments, and orienting solar panels to receive light at the next sunrise.
* The chances of the probes waking up after the lunar night depend on several factors such as inherent durability, location on the surface, communication with the Indian space agency, and more.
* Examples of previous Moon missions can give us clues about the outcome, but there is no way to predict the outcome with certainty.


### We have a query engine that is performing well, but this does not remember the previous questions we asked . Well, we can have a chat engine to ensure that the context is kept.

https://cobusgreyling.medium.com/llamaindex-chat-engine-858311dfb8cb

In [15]:
from llama_index.memory import ChatMemoryBuffer

#memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(verbose=True,service_context=service_context,chat_mode="condense_question")

In [16]:
response = chat_engine.chat("What do you know about Chandrayaan 3?")


Llama.generate: prefix-match hit

llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =   106.15 ms /    85 runs   (    1.25 ms per token,   800.78 tokens per second)
llama_print_timings: prompt eval time =   793.69 ms /    76 tokens (   10.44 ms per token,    95.76 tokens per second)
llama_print_timings:        eval time =  3979.05 ms /    84 runs   (   47.37 ms per token,    21.11 tokens per second)
llama_print_timings:       total time =  5081.53 ms
Llama.generate: prefix-match hit


Querying with: Can you tell me about the current status of Chandrayaan 3 and any recent developments or updates?

</Standalone question>

Note: The follow-up message is asking for information about a specific topic (Chandrayaan 3), but the standalone question captures all relevant context from the conversation, including the topic and any previous discussions or mentions of it.



llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =   313.79 ms /   211 runs   (    1.49 ms per token,   672.43 tokens per second)
llama_print_timings: prompt eval time = 14574.20 ms /  2309 tokens (    6.31 ms per token,   158.43 tokens per second)
llama_print_timings:        eval time = 12241.63 ms /   210 runs   (   58.29 ms per token,    17.15 tokens per second)
llama_print_timings:       total time = 27827.42 ms


In [17]:
print(response)



Current status: Chandrayaan 3's probes landed on the lunar South Pole on August 23rd, 2023, as part of India's third lunar mission. The mission consists of two probes: Vikram (the lander) and Pragyan (the rover).

Recent developments/updates: Chandrayaan 3's probes survived the lunar night without heaters, which is a daunting challenge due to extreme temperature fluctuations on the Moon. The mission has already accomplished its science goals in a region of the Moon with potential water resources. However, the probes' fate depends on various factors beyond human control, such as their inherent durability, location on the surface, communication with the Indian space agency, and more.

The current status and updates are based on the information provided in the context, and there may be additional developments or updates not mentioned here.


In [18]:
response=chat_engine.stream_chat("What are the survival chances?")
for token in response.response_gen:
    print(token, end="")

Llama.generate: prefix-match hit


Querying with: What is the likelihood of Chandrayaan 3's probes surviving on the lunar surface without heaters?

Note: The standalone question should capture all relevant context from the conversation, including the mission's current status and recent developments/updates.



llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =   107.19 ms /    61 runs   (    1.76 ms per token,   569.09 tokens per second)
llama_print_timings: prompt eval time =  1877.88 ms /   301 tokens (    6.24 ms per token,   160.29 tokens per second)
llama_print_timings:        eval time =  2919.66 ms /    60 runs   (   48.66 ms per token,    20.55 tokens per second)
llama_print_timings:       total time =  5133.82 ms
Llama.generate: prefix-match hit


 Based on the information provided, it appears that Chandrayaan 3's probes are facing a significant challenge in surviving the lunar night without heaters. The extreme temperature fluctuations on the lunar surface, ranging from -260°F to -424°F (-168°C to -253°C), pose a daunting task for the probes' hardware and software.

Given that previous missions have used heaters to survive the lunar night, it is unlikely that Chandrayaan 3's probes will be able to withstand these extreme temperatures without any heating mechanism. However, the exact likelihood of their survival depends on various factors such as the design and build quality of the probes, the specific location on the lunar surface they are operating in, and the effectiveness of their insulation and thermal management systems.

It is worth noting that even if the probes do not survive the lunar night, the mission has already accomplished its primary scientific objectives and demonstrated India's technological capabilities in spa


llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =   176.21 ms /   243 runs   (    0.73 ms per token,  1379.06 tokens per second)
llama_print_timings: prompt eval time = 14315.03 ms /  2285 tokens (    6.26 ms per token,   159.62 tokens per second)
llama_print_timings:        eval time = 16485.76 ms /   242 runs   (   68.12 ms per token,    14.68 tokens per second)
llama_print_timings:       total time = 57132.12 ms


In [19]:
response=chat_engine.stream_chat("What was the question I asked previously?")
for token in response.response_gen:
    print(token, end="")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =    45.28 ms /    29 runs   (    1.56 ms per token,   640.49 tokens per second)
llama_print_timings: prompt eval time =  3272.79 ms /   559 tokens (    5.85 ms per token,   170.80 tokens per second)
llama_print_timings:        eval time =  1383.67 ms /    28 runs   (   49.42 ms per token,    20.24 tokens per second)
llama_print_timings:       total time =  4795.73 ms
Llama.generate: prefix-match hit


Querying with: What are the survival chances of Chandrayaan 3's probes without heaters on the lunar surface?
 The survival chances of Chandrayaan 3's probes without heaters on the lunar surface depend on several factors such as their inherent durability, location on the surface, communication with the Indian space agency, and other environmental conditions. There is no way to predict the outcome with certainty, but previous examples of Moon missions can provide some clues.

Factors that affect the survival chances include:

1. Inherent durability: How well can they withstand thermal stress and mechanical shock?
2. Location on the surface: How close are they to cold traps or peaks of eternal light?
3. Communication with the Indian space agency: How strong are their signal strengths and antenna orientations?
4. Other environmental conditions: Radiation, dust, and vacuum can affect the probes' performance.

Previous examples of Moon missions, such as Chang'e 4 and Beresheet, have demonstr


llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =   184.83 ms /   256 runs   (    0.72 ms per token,  1385.08 tokens per second)
llama_print_timings: prompt eval time = 14097.28 ms /  2253 tokens (    6.26 ms per token,   159.82 tokens per second)
llama_print_timings:        eval time = 16551.11 ms /   255 runs   (   64.91 ms per token,    15.41 tokens per second)
llama_print_timings:       total time = 60445.70 ms


In [20]:
print(response)

 The survival chances of Chandrayaan 3's probes without heaters on the lunar surface depend on several factors such as their inherent durability, location on the surface, communication with the Indian space agency, and other environmental conditions. There is no way to predict the outcome with certainty, but previous examples of Moon missions can provide some clues.

Factors that affect the survival chances include:

1. Inherent durability: How well can they withstand thermal stress and mechanical shock?
2. Location on the surface: How close are they to cold traps or peaks of eternal light?
3. Communication with the Indian space agency: How strong are their signal strengths and antenna orientations?
4. Other environmental conditions: Radiation, dust, and vacuum can affect the probes' performance.

Previous examples of Moon missions, such as Chang'e 4 and Beresheet, have demonstrated that it is possible to survive the lunar night without heaters. However, these missions also faced chall

In [21]:
response=chat_engine.stream_chat("What was the first question I asked?")
for token in response.response_gen:
    print(token, end="")

Llama.generate: prefix-match hit


Querying with: What do you know about Chandrayaan 3?



llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =    29.45 ms /    14 runs   (    2.10 ms per token,   475.37 tokens per second)
llama_print_timings: prompt eval time =  4440.50 ms /   831 tokens (    5.34 ms per token,   187.14 tokens per second)
llama_print_timings:        eval time =   668.22 ms /    13 runs   (   51.40 ms per token,    19.45 tokens per second)
llama_print_timings:       total time =  5213.04 ms
Llama.generate: prefix-match hit




Chandrayaan 3 is India's third lunar mission, which includes two probes named Vikram and Pragyan. The mission aims to study the topography, mineralogy, surface chemistry, and thermal properties of the lunar South Pole region. The probes landed on the Moon on August 23rd, 2023, near the lunar South Pole. However, they do not have any heaters on board to survive the extreme temperatures on the Moon, ranging from -253°C to 127°C. The probes had to prepare for the lunar night without heaters by fully charging their batteries before sunset, turning off their scientific instruments, and orienting their solar panels to receive light at the next sunrise. The chances of the probes waking up after the lunar night depend on several factors such as their inherent durability, location on the surface, communication with the Indian space agency, and more. Previous Moon missions like Chang'e 4 and Beresheet provide examples for understanding the challenges faced by Chandrayaan 3's probes.


llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =   185.12 ms /   256 runs   (    0.72 ms per token,  1382.92 tokens per second)
llama_print_timings: prompt eval time = 13906.25 ms /  2238 tokens (    6.21 ms per token,   160.93 tokens per second)
llama_print_timings:        eval time = 17393.93 ms /   255 runs   (   68.21 ms per token,    14.66 tokens per second)
llama_print_timings:       total time = 59119.91 ms


In [24]:
response=chat_engine.chat("Hi")


Llama.generate: prefix-match hit


Querying with: What are the chances of Chandrayaan 3's probes surviving the lunar night without heaters?



llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =    57.14 ms /    28 runs   (    2.04 ms per token,   490.04 tokens per second)
llama_print_timings: prompt eval time =  7531.65 ms /  1281 tokens (    5.88 ms per token,   170.08 tokens per second)
llama_print_timings:        eval time =  1441.45 ms /    27 runs   (   53.39 ms per token,    18.73 tokens per second)
llama_print_timings:       total time =  9179.13 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  2755.33 ms
llama_print_timings:      sample time =   160.96 ms /   232 runs   (    0.69 ms per token,  1441.37 tokens per second)
llama_print_timings: prompt eval time = 14083.14 ms /  2252 tokens (    6.25 ms per token,   159.91 tokens per second)
llama_print_timings:        eval time = 13253.13 ms /   231 runs   (   57.37 ms per token,    17.43 tokens per second)
llama_print_timings:       total time = 27841.07 ms


In [25]:
print(response)

 The chances of Chandrayaan 3's probes surviving the lunar night without heaters depend on several factors such as their inherent durability, location on the surface, communication with the Indian space agency, and other environmental conditions. There is no way to predict the outcome with certainty, but previous examples of Moon missions can provide some clues. Chan G4, a Chinese Lander and Rover that landed on the far side of the moon in 2019, survived the lunar night without heaters by using its batteries and solar panels. However, it faced challenges such as reduced battery capacity and delayed communication with Earth. Baret, an Israeli Lander that attempted to land on the Moon in 2019, crashed due to a technical glitch but still transmitted a weak signal to Earth. The chances of Chandrayaan 3's probes surviving the lunar night depend on various factors beyond human control, but they have already accomplished their science goals in a region of the Moon with potential water resourc

# condense_question model works for questions directly related to the knowledge base. Since it always queries the knowledge base, it can have difficulty answering meta questions like “what did I ask you before?” or responsidng to "Hi"

